In [ ]:
import numpy as np
from scipy.sparse import kron, eye, csr_matrix, dia_matrix
from scipy.linalg import eigh
from scipy.linalg import expm

class CDFQA:
    def __init__(self):
        pass

    
    def fast_expm(self, H, convergence_criteria=1e-6, non_zero_tol=1e-14):
        """
        Fast matrix exponential for sparse or dense matrices.
        Converts sparse matrices to dense before applying expm.
        """
        if isinstance(H, (csr_matrix, dia_matrix)):
            H = H.toarray()  # Convert sparse matrix to dense
        P = expm(H)  # Apply expm to dense matrix
        P[np.abs(P) < non_zero_tol] = 0  # Eliminate small elements
        return P.astype(np.complex128)
    
    def ham_nn(self, N, spin1, spin2, couplingNN, perBC):
        """
        Generates the Nearest Neighbor (NN) Hamiltonian.
        """
        def get_spin_matrix(spin):
            if spin == 0:
                return np.eye(2, dtype=np.complex128)  # Ensure complex dtype
            elif spin == 1:
                return np.array([[0, 1], [1, 0]], dtype=np.complex128)  # Pauli-X
            elif spin == 2:
                return np.array([[0, -1j], [1j, 0]], dtype=np.complex128)  # Pauli-Y
            else:
                return np.array([[1, 0], [0, -1]], dtype=np.complex128)  # Pauli-Z

        s11 = get_spin_matrix(spin1)
        s22 = get_spin_matrix(spin2)
        s1 = csr_matrix(s11, dtype=np.complex128)
        s2 = csr_matrix(s22, dtype=np.complex128)

        H = csr_matrix((2**N, 2**N), dtype=np.complex128)  # Ensure complex initialization

        for i in range(N - 1):
            if i == 0:
                H += couplingNN * kron(kron(s1, s2), eye(2**(N-2), dtype=np.complex128))
            elif i == N - 2:
                H += couplingNN * kron(eye(2**(N-2), dtype=np.complex128), kron(s1, s2))
            else:
                H += couplingNN * kron(eye(2**i, dtype=np.complex128), kron(kron(s1, s2), eye(2**(N-2-i), dtype=np.complex128)))
        
        if perBC == 1:
            H += couplingNN * kron(kron(s2, eye(2**(N-2), dtype=np.complex128)), s1)

        return H

    def ham_onsite(self, N, spin1, couplingNN):
        """
        Generates the Onsite Hamiltonian.
        """
        def get_spin_matrix(spin):
            if spin == 0:
                return np.eye(2, dtype=np.complex128)  # Ensure complex dtype
            elif spin == 1:
                return np.array([[0, 1], [1, 0]], dtype=np.complex128)  # Pauli-X
            elif spin == 2:
                return np.array([[0, -1j], [1j, 0]], dtype=np.complex128)  # Pauli-Y
            else:
                return np.array([[1, 0], [0, -1]], dtype=np.complex128)  # Pauli-Z

        s1 = get_spin_matrix(spin1)
        H = np.zeros((2**N, 2**N), dtype=np.complex128)  # Ensure the matrix is initialized as complex

        for i in range(N):
            if i == 0:
                H += np.kron(s1, np.eye(2**(N-1), dtype=np.complex128))
            elif i == N - 1:
                H += np.kron(np.eye(2**(N-1), dtype=np.complex128), s1)
            else:
                H += np.kron(np.eye(2**i, dtype=np.complex128), np.kron(s1, np.eye(2**(N-i-1), dtype=np.complex128)))

        return couplingNN * H

    def cdfqa_energy_vs_cir_depth(self, N, HP, HM, HCD, psi_initial, Layers, dt):
        """
        Computes energy vs circuit depth for the given system.
        """
        EN = np.zeros(Layers, dtype=np.complex128)  # Initialize complex
        B = np.zeros(Layers, dtype=np.complex128)  # Initialize complex
        G = np.zeros(Layers, dtype=np.complex128)  # Initialize complex
        CM = HM @ HP - HP @ HM
        CMCD = HCD @ HP - HP @ HCD

        psi = psi_initial.astype(np.complex128)  # Ensure psi is complex
        EN[0] = np.conjugate(psi).T @ HP @ psi
        B[0] = 0
        G[0] = 0

        for i in range(1, Layers):
            psi = self.fast_expm(-1j * dt * G[i-1] * HCD) @ (
                self.fast_expm(-1j * dt * B[i-1] * HM) @ (
                    self.fast_expm(-1j * dt * HP) @ psi
                )
            )

            B[i] = -1j * (np.conjugate(psi).T @ CM @ psi)
            G[i] = -1j * (np.conjugate(psi).T @ CMCD @ psi)
            EN[i] = np.conjugate(psi).T @ HP @ psi

        return EN, B, G

    def cdfqa_population_vs_cir_depth(self, N, HP, HM, HCD, psi_initial, Layers, dt):
        """
        Computes population vs circuit depth for the given system.
        """
        # Initialize B, G, and population arrays
        B = np.zeros(Layers, dtype=np.complex128)
        G = np.zeros(Layers, dtype=np.complex128)
        population = np.zeros((2**N, Layers), dtype=np.complex128)

        # Commutators
        CM = HM @ HP - HP @ HM
        CMCD = HCD @ HP - HP @ HCD

        # Eigen decomposition of HP
        D, EV = eigh(HP)

        # Initial state
        B[0] = 0
        G[0] = 0
        psi = psi_initial.astype(np.complex128)

        # First population calculation
        population[:, 0] = np.abs(np.dot(np.conjugate(psi).T, EV).T)**2

        # Iterate over layers
        for i in range(1, Layers):
            # Time evolution for current layer
            psi = self.fast_expm(-1j * dt * G[i-1] * HCD, dt) @ (
                self.fast_expm(-1j * dt * B[i-1] * HM, dt) @ (
                    self.fast_expm(-1j * dt * HP, dt) @ psi
                )
            )

            # Update B and G
            B[i] = -1j * (np.conjugate(psi).T @ CM @ psi)
            G[i] = -1j * (np.conjugate(psi).T @ CMCD @ psi)

            # Update population
            population[:, i] = np.abs(np.dot(np.conjugate(psi).T, EV).T)**2

        return population
